<a href="https://colab.research.google.com/github/Moiz-36/Car-Price-Prediction/blob/main/fake_new_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

  Data preprocessing


In [ ]:
new_data = pd.read_csv('FA-KES-Dataset.csv', encoding='latin-1')

In [ ]:
new_data.shape

(804, 7)

In [ ]:
new_data.head()

,unit_id,article_title,article_content,source,date,location,labels
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,4/5/2017,idlib,0
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,nna,4/7/2017,homs,0
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,nna,4/16/2017,aleppo,0
3,1914947534,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,nna,4/19/2017,aleppo,0
4,1914947535,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,nna,7/10/2016,aleppo,0


In [ ]:
new_data.isnull().sum()

,0
unit_id,0
article_title,0
article_content,0
source,0
date,0
location,0
labels,0


In [ ]:
new_data['content']=new_data['article_title']+' '+new_data['article_content']

In [ ]:
print(new_data['content'])

0      Syria attack symptoms consistent with nerve ag...
1      Homs governor says U.S. attack caused deaths b...
2      Death toll from Aleppo bomb attack at least 11...
3      Aleppo bomb blast kills six Syrian state TV We...
4      29 Syria Rebels Dead in Fighting for Key Alepp...
                             ...                        
799    Turkish Bombardment Kills 20 Civilians in Syri...
800    Martyrs as Terrorists Shell Aleppos Salah Eddi...
801    Chemical Attack Kills Five Syrians in Aleppo S...
802    5 Killed as Russian Military Chopper Shot down...
803    Syrian Army Kills 48 ISIL Terrorists in Deir E...
Name: content, Length: 804, dtype: object


In [ ]:
x=new_data.drop(columns='labels',axis=1)
y=new_data['labels']

In [ ]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[PorterStemmer().stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [ ]:
new_data['content']=new_data['content'].apply(stemming)

In [ ]:
x=new_data['content'].values
y=new_data['labels'].values

In [ ]:
vectorizer=TfidfVectorizer(ngram_range=(1,2), max_features=10000, stop_words='english')


x = vectorizer.fit_transform(new_data['content'].values)

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [ ]:
model=LogisticRegression()

In [ ]:
model.fit(xtrain,ytrain)

LogisticRegression()

In [ ]:
x_train_prediction=model.predict(xtrain)
training_data_accuracy=accuracy_score(x_train_prediction,ytrain)

In [ ]:
print(training_data_accuracy)

0.9097978227060654


In [ ]:
x_test_prediction=model.predict(xtest)
test_data_accuracy=accuracy_score(x_test_prediction,ytest)

In [ ]:
print(test_data_accuracy)

0.5527950310559007


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, test_preds))

              precision    recall  f1-score   support

           0       0.31      0.05      0.09        76
           1       0.51      0.89      0.65        85

    accuracy                           0.50       161
   macro avg       0.41      0.47      0.37       161
weighted avg       0.42      0.50      0.39       161



In [ ]:
from sklearn.naive_bayes import MultinomialNB
modelp = MultinomialNB()
modelp.fit(xtrain, ytrain)

train_preds = modelp.predict(xtrain)
test_preds = modelp.predict(xtest)

from sklearn.metrics import accuracy_score
print("Train Accuracy:", accuracy_score(train_preds, ytrain))
print("Test Accuracy:", accuracy_score(test_preds, ytest))

Train Accuracy: 0.7153965785381027
Test Accuracy: 0.5031055900621118
